<a href="https://colab.research.google.com/github/OlesiaTvardovska/DataInjestionService/blob/dev/News_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#News Classification with LSTM using TensorFlow 2.0 and Keras

In [2]:
#Libraries import
import nltk
nltk.download('stopwords')
import csv
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords

from tensorflow import keras
from keras import metrics
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# Declaring hyperparameters for the LSTM
vocabulary = 5000
embedding_dim = 64
max_length = 200
trunc_type = 'post'
padding_type = 'post'
oovVal = 'unseenWord'
percentOfTrainData = .8

In [4]:
# Read input data to the DataFrame
train=pd.read_csv('drive/My Drive/ValidDataCsv3.csv')
train = train[['headline_tok', ' is_arg', ' is_emo_arg_mult',' real_url']]
train[train[' is_arg']==''] = 0
train[train[' is_emo_arg_mult']==''] = 0
train = train.dropna(subset=[' is_arg', ' is_emo_arg_mult'])
train.info()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,8,9,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 54807 entries, 0 to 84590
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   headline_tok      54806 non-null  object
 1    is_arg           54807 non-null  object
 2    is_emo_arg_mult  54807 non-null  object
 3    real_url         50339 non-null  object
dtypes: object(4)
memory usage: 2.1+ MB


In [5]:
#Check data has been read in properly

train.head()

,headline_tok,is_arg,is_emo_arg_mult,real_url
0,Школи з мовою навчання національних меншин зак...,,,http://hvylya.net/news/digest/shkoli-z-movoyu...
1,Украинская правда извинилась за распростране...,,,http://hvylya.net/news/digest/ukrainskaya-pra...
2,Українських школярів будуть навчати фінансової...,,,http://hvylya.net/news/digest/ukrayinskih-shk...
3,Військові пенсіонери 20 січня перекриватимуть ...,,,http://hvylya.net/news/digest/viyskovi-pensio...
4,Як обстежуватимуть житло претендентів на субсидії,,,http://hvylya.net/news/digest/yak-obstezhuvat...


In [6]:
# The fitting column contains only headline
train['total']=train['headline_tok']#+train[' is_other']
stop = ["a",
"б","в","г","е","ж","з","м","т","у","я","є","і","аж","ви",
"де","до","за","зі","ми","на","не","ну","із", "весь", "свій", "мати",
"нх","ні","по","та","ти","то","ту","ті","що","як", "через",
"це","цю","ця","ці","чи","ще","що","як","їй","їм",
"їх","її","або","але","ало","без","був","вам","вас","бути",
"ваш","вже","все","всю","вся","від","він","два","дві","для",
"ким","мож","моя","моє","мої","міг","між","мій","над","нам","нас","наш","нею","неї","них","ніж","ній","ось",
"при","про","під","пір","раз","рік","сам","сих","сім","так","там", "який", "такий", "багато","цей","той", "якщо","bbcccnn", "ua"
"будемо","більш","вгору","вміти","внизу","вісім",
"давно","даром","добре","довго","друго","дякую","життя","зараз","знову","какая","кожен","кожна","кожне","кожні","краще","ледве",
"майже","менше","могти","можна","назад",
"немає","нижче","нього","однак","п'ять","перед","поруч","потім","проти","після","років","самим","самих","самій","свого","своєї","своїх","собою","справ","такий","також","тепер","тисяч","тобою","треба",
"трохи","усюди","усіма","хочеш","цього","цьому","часто","через","шість","якого","іноді","інший","інших","будете","будуть","більше","всього","всьому","далеко","десять","досить","другий","дійсно","завжди","звідси","зовсім",
"кругом","кілька","людина","можуть","навіть","навіщо","нагорі","небудь","низько","ніколи","нікуди","нічого","обидва","одного","однієї","п'ятий","перший"]
train['total'] = train['total'].astype(str)
train['total'] = train['total'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#!pip install git+https://github.com/kmike/pymorphy2.git@master

!pip install pymorphy2-dicts-uk
import pymorphy2
import pymorphy2_dicts_uk

mm = pymorphy2.MorphAnalyzer(lang="uk")
train = train[:5000]
train['total'] = train['total'].apply(lambda x: ' '.join([mm.parse(word)[0].normal_form for word in x.split() if word not in (stop)]))
train['total'] = train['total'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

     |████████████████████████████████| 5.0MB 7.6MB/s 


ModuleNotFoundError: ignored

In [ ]:
# Create two arrays for texts and labels(true/fake)
textsArray = []
labels = []

textsArray = train['total'].values.tolist()
labels = train[' is_emo_arg_mult'].values.tolist()

print(len(labels))
print(len(textsArray))

In [ ]:
# Data preprocesssing
labels = [item.replace(" ", "") for item in labels]
labels = ["0" if x == '' else x for x in labels]
labels = [float(i) for i in labels] 
labelsStr = ["like" if x < 0.5 else "dislike" for x in labels]

train[' is_emo_arg_mult']  = labelsStr
print(len(labelsStr))

In [ ]:
#Split data into train and test dataset
size = int(len(textsArray) * percentOfTrainData)

trainData = textsArray[0: size]
trainLabels = labelsStr[0: size]

testData = textsArray[size:]
testLabels = labelsStr[size:]

print(len(trainData))
print(len(testData))

In [ ]:
tokenizer = Tokenizer(num_words = vocabulary, oov_token=oovVal)
tokenizer.fit_on_texts(trainData)
word_index = tokenizer.word_index
dict(list(word_index.items())[0:10])

In [ ]:
# Turn tokens into lists of sequence
print(trainData[3])
trainSequences = tokenizer.texts_to_sequences(trainData)
print(trainSequences[3])

In [ ]:
# we need sequences in the same size, so we use padding
trainList = pad_sequences(trainSequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(len(trainSequences[0]))
print(len(trainList[0]))

In [ ]:
print(trainList[3])

In [ ]:
# Do the same transformations for the test dataset
testSequences = tokenizer.texts_to_sequences(testData)
testList = pad_sequences(testSequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(len(testSequences))
print(testList.shape)

In [ ]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labelsStr)

trainingSeq = np.array(label_tokenizer.texts_to_sequences(trainLabels))
testSeq = np.array(label_tokenizer.texts_to_sequences(testLabels))
print(trainingSeq[0])
print(trainingSeq.shape)

print(testSeq[0])
print(testSeq.shape)

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_article(text):
    return ' '.join([reverse_word_index.get(i, '') for i in text])
print(decode_article(trainList[0]))
print(trainData[0])

In [ ]:
model = tf.keras.Sequential([
    # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
    tf.keras.layers.Embedding(vocabulary, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
#    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])
model.summary()

In [ ]:
!pip install keras-metrics
import keras_metrics
class_weight = {0: 1.,
                1: 50.,
                2: 2.}
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
num_epochs = 1
history = model.fit(trainList, trainingSeq, epochs=num_epochs,
                     validation_data=(testList, testSeq))

In [ ]:
classes=[1,2]
y_pred=model.predict_classes(testList)
con_mat = tf.math.confusion_matrix(labels=testSeq, predictions=y_pred).numpy()
TP = tf.math.count_nonzero(y_pred * testSeq)
TN = tf.math.count_nonzero((y_pred - 2) * (testSeq - 2))
FP = tf.math.count_nonzero(y_pred * (testSeq - 1))
FN = tf.math.count_nonzero((y_pred - 2) * testSeq)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2 * precision * recall / (precision + recall)
print(recall)
print(precision)
print(f1)

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2 * precision * recall / (precision + recall)![alt text](https://)